# Lab 2022-04-22: Satisfiability Modulo Theories

During this lab you will be using Z3 solver to deduce facts and solve combinatorial problems.

There are two ways to use Z3, either via the [SMT-LIB](https://jfmc.github.io/z3-play/) language or via a [programmatic APIs](https://z3prover.github.io/api/html/index.html). First we'll see an example of the SMT-LIB interface, but we'll focus on the Python API. The material covered in the [Z3Py tutorial](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) is enough for our purposes. For more examples and advanced usage you can look at the [SAT/SMT by Example](https://sat-smt.codes/SAT_SMT_by_example.pdf) book.


## Using Z3 in Jupyter

As an example we can use the solver to verify that, within the boolean logic, if $p\Rightarrow q$ and $q\Rightarrow r$, then $p\Rightarrow r$ (transitivity of implication). This can be verified by showing that the formula $$((p\Rightarrow q) \land (q\Rightarrow r)) \Rightarrow (p\Rightarrow r)$$ is a tautology: for any value True/False assigned to $ p, q , r$ the formula is true; or by checking that its negation $$\neg(((p\Rightarrow q) \land (q\Rightarrow r)) \Rightarrow (p\Rightarrow r))$$ cannot be satisfied.




### SMT-LIB

Code cells are executed by the selected kernel (Python 3, based on [IPython](https://ipython.readthedocs.io/en/stable/)), which provides [Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to a better interaction with the environment. In particular we'll use `%%script` command which works like the [Unix Shebang](https://en.wikipedia.org/wiki/Shebang_(Unix)) passing the content of the cell to the executing program as standard input.

In [2]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(assert (not (=> (and (=> p q) (=> q r))
                 (=> p r))))
(check-sat)

unsat


The converse is not true, that is $$(p\Rightarrow r) \Rightarrow ((p\Rightarrow q) \land (q\Rightarrow r))$$ is not a tautology:

In [3]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(assert (not (=> (=> p r)
                 (and (=> p q) (=> q r)))))
(check-sat)

sat


### Python API

Let's see the same example using the Python API:

In [4]:
from z3 import *

p, q, r = Bools('p q r')
print (type (p))

s = Solver()

s.add(Not(Implies(And(Implies(p, q), Implies(q, r)), Implies(p, r))))
print(s.check())


<class 'z3.z3.BoolRef'>
unsat


Note that here we import the whole `z3` namespace, which is fine for small examples; but in general it might clash with names defined in your code.

Let's see the converse

In [5]:
from z3 import *

p, q, r = Bools('p q r')

s = Solver()

s.add(Not(Implies(Implies(p, r), And(Implies(p, q), Implies(q, r)))))
print(s.check())

sat


since the negation of the formula is satisfiable, it's not a tautology. We can look at the model for a counterexample:

In [6]:
print(f'A counterexample: {s.model()}')
for v in [p, q, r]:
    print(f'The value of {v} is {s.model()[v]}')


A counterexample: [p = True, q = False, r = True]
The value of p is True
The value of q is False
The value of r is True


## Z3 as SAT solver

Z3 can be used to solve Boolean Satisfiability (SAT) problems. 



### Unicorn example

Consider the unicorn example introduced during the lecture:

-   If the unicorn is mythical, then it is immortal
-   If the unicorn is not mythical, then it is a mortal mammal
-   If the unicorn is either immortal or a mammal, then it is horned
-   The unicorn is magical if it is horned

Abstracting the domain using propositional variables

-   $m$: the unicorn is mythical
-   $i$: the unicorn is immortal
-   $l$: the unicorn is mammal
-   $h$: the unicorn is horned
-   $g$: the unicorn is magical

the above statements became

$$(m\rightarrow i)\wedge(\neg m \rightarrow (\neg i \wedge l))\wedge((i \vee l) \rightarrow h)\wedge(h\rightarrow g)$$

The corresponding assertions are:

In [7]:
from z3 import *

m, i, l, h, g = Bools('m i l h g')

s = Solver()

s.add(Implies(m, i))
s.add(Implies(Not(m), And(Not(i), l)))
s.add(Implies(Or(i, l), h))
s.add(Implies(h, g))

We can check that our assumptions are not contradictory:

In [8]:
s.check()

sat

Now we can answer some questions about the domain.

1.  The unicorn must be mythical?

To answer that we use a statement that negate the question, but that's just an assumption that we use to verify the property and then retract. To this end we can use the `push`, `pop` methods of the solver (more details in [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-scopes)). The first creates a nested context and the second deletes it by retracting all assertions added after the push. So let's check whether the unicorn cannot be mythical (e.g., $m$ can be false):

In [9]:
print(s.assertions())

s.push()
s.add(Not(m))
print(s.assertions())
print(s.check())
s.pop()


s.push()
s.add(m)
print(s.assertions())
print(s.check())
s.pop()

print(s.assertions())

[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(m)]
sat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 m]
sat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


As you see $m$ can be both true or false, so we cannot tell anything about the unicorn being mythical. Now verify the other properties:

2.  Must be magical?

In [10]:
print(s.assertions())

s.push()
s.add(Not(g))
print(s.assertions())
print(s.check())
s.pop()


s.push()
s.add(g)
print(s.assertions())
print(s.check())
s.pop()

print(s.assertions())

[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(g)]
unsat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 g]
sat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


3.  Must be horned?

In [11]:
print(s.assertions())

s.push()
s.add(Not(h))
print(s.assertions())
print(s.check())
s.pop()


s.push()
s.add(h)
print(s.assertions())
print(s.check())
s.pop()

print(s.assertions())

[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 Not(h)]
unsat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g),
 h]
sat
[Implies(m, i),
 Implies(Not(m), And(Not(i), l)),
 Implies(Or(i, l), h),
 Implies(h, g)]


### Graph colouring

In the previous exercise we looked at SAT as a way to answer to questions; i.e. by focusing on logical deduction. However SAT can be used to find solutions to combinatorial (NP) problems. In this exercise we will consider the [Graph Colouring problem](https://en.wikipedia.org/wiki/Graph_coloring).

As seen in the lecture Graph Colouring can be used to solve different problems (e.g. [scheduling](https://en.wikipedia.org/wiki/Graph_coloring#Scheduling)).

Consider the following problem by James L. Hein:

> Some people form committees to do various tasks. The problem is to schedule the committee meetings in as few time slots as possible.
> We'll represent each person with a number. For example, let $S = \{1, 2, 3, 4, 5, 6, 7\}$ represent a set of seven people, and suppose they have formed six three-person committees as follows:
>
> $$S_1 = \{1, 2, 3\}, S_2 = \{2, 3, 4\}, S_3 = \{3, 4, 5\}, S_4 = \{4, 5, 6\}, S_5 = \{5, 6, 7\}, S_6 = \{1, 6, 7\}$$
>
> We can model the problem with a graph, where the committee names are the vertices and an edge connects two vertices if a person belongs to both committees represented by the vertices.
> If each committee meets for one hour, what is the smallest number of hours needed for the committees to do their work?
> From the graph, it follows that an edge between two committees means that they have at least one member in common. Thus, they cannot meet at the same time. No edge between committees means that they can meet at the same time. For example, committees $S_1$ and $S_4$ can meet at the same hour.

Solve the problem using a graph colouring problem with Z3. Note that variables can be also created by means of Python comprehension:

```python
V = tuple(Bool(f's_{i}') for i in range(7))

s = Solver()
s.add(Or(V[0], V[1]))
s.assertions()
```

note the `Bool` method (without the `s` at the end).

In [12]:
# One time slot
# Not satisfied
V = tuple(Bool(f's_{i+1}') for i in range(7))

s = Solver()

s.add(V[1])
s.add(And(V[1], Not(Or(V[2], V[3], V[6]))))

s.add(V[2])
s.add(And(V[2], Not(Or(V[1], V[3], V[4]))))

s.add(V[3])
s.add(And(V[3], Not(Or(V[1], V[2], V[4], V[5]))))

s.add(V[4])
s.add(And(V[4], Not(Or(V[2], V[3], V[5], V[6]))))

s.add(V[5])
s.add(And(V[5], Not(Or(V[3], V[4], V[6]))))

s.add(V[6])
s.add(And(V[6], Not(Or(V[1], V[4], V[5]))))

print(s.check())
s.assertions()

unsat


[s_2,
 And(s_2, Not(Or(s_3, s_4, s_7))),
 s_3,
 And(s_3, Not(Or(s_2, s_4, s_5))),
 s_4,
 And(s_4, Not(Or(s_2, s_3, s_5, s_6))),
 s_5,
 And(s_5, Not(Or(s_3, s_4, s_6, s_7))),
 s_6,
 And(s_6, Not(Or(s_4, s_5, s_7))),
 s_7,
 And(s_7, Not(Or(s_2, s_5, s_6)))]

In [13]:
# Two time slots
# Not satisfied
V = [[Bool(f's_{j}_{i}') for i in range(3)] for j in range(7)]
s = Solver()

s.add(Or(V[1][1], V[1][2]))
s.add(And(V[1][1], Not(Or(V[2][1], V[3][1], V[6][1]))))
s.add(And(V[1][2], Not(Or(V[2][2], V[3][2], V[6][2]))))

s.add(Or(V[2][1], V[2][2]))
s.add(And(V[2][1], Not(Or(V[1][1], V[3][1], V[4][1]))))
s.add(And(V[2][2], Not(Or(V[1][2], V[3][2], V[4][2]))))

s.add(Or(V[3][1], V[3][2]))
s.add(And(V[3][1], Not(Or(V[1][1], V[2][1], V[4][1], V[5][1]))))
s.add(And(V[3][2], Not(Or(V[1][2], V[2][2], V[4][2], V[5][2]))))

s.add(Or(V[4][1], V[4][2]))
s.add(And(V[4][1], Not(Or(V[2][1], V[3][1], V[5][1], V[6][1]))))
s.add(And(V[4][2], Not(Or(V[2][2], V[3][2], V[5][2], V[6][2]))))

s.add(Or(V[5][1], V[5][2]))
s.add(And(V[5][1], Not(Or(V[3][1], V[4][1], V[6][1]))))
s.add(And(V[5][2], Not(Or(V[3][2], V[4][2], V[6][2]))))

s.add(Or(V[6][1], V[6][2]))
s.add(And(V[6][1], Not(Or(V[1][1], V[4][1], V[5][1]))))
s.add(And(V[6][2], Not(Or(V[1][2], V[4][2], V[5][2]))))

print(s.check())
s.assertions()

unsat


[Or(s_1_1, s_1_2),
 And(s_1_1, Not(Or(s_2_1, s_3_1, s_6_1))),
 And(s_1_2, Not(Or(s_2_2, s_3_2, s_6_2))),
 Or(s_2_1, s_2_2),
 And(s_2_1, Not(Or(s_1_1, s_3_1, s_4_1))),
 And(s_2_2, Not(Or(s_1_2, s_3_2, s_4_2))),
 Or(s_3_1, s_3_2),
 And(s_3_1, Not(Or(s_1_1, s_2_1, s_4_1, s_5_1))),
 And(s_3_2, Not(Or(s_1_2, s_2_2, s_4_2, s_5_2))),
 Or(s_4_1, s_4_2),
 And(s_4_1, Not(Or(s_2_1, s_3_1, s_5_1, s_6_1))),
 And(s_4_2, Not(Or(s_2_2, s_3_2, s_5_2, s_6_2))),
 Or(s_5_1, s_5_2),
 And(s_5_1, Not(Or(s_3_1, s_4_1, s_6_1))),
 And(s_5_2, Not(Or(s_3_2, s_4_2, s_6_2))),
 Or(s_6_1, s_6_2),
 And(s_6_1, Not(Or(s_1_1, s_4_1, s_5_1))),
 And(s_6_2, Not(Or(s_1_2, s_4_2, s_5_2)))]

In [14]:
# Three time slots
# Satisfied
V = [[Bool(f's_{j}_{i}') for i in range(4)] for j in range(7)]
s = Solver()

s.add(Or(V[1][1], V[1][2], V[1][3]))
s.add(Implies(V[1][1], Not(Or(V[2][1], V[3][1], V[6][1]))))
s.add(Implies(V[1][2], Not(Or(V[2][2], V[3][2], V[6][2]))))
s.add(Implies(V[1][3], Not(Or(V[2][3], V[3][3], V[6][3]))))

s.add(Or(V[2][1], V[2][2], V[2][3]))
s.add(Implies(V[2][1], Not(Or(V[1][1], V[3][1], V[4][1]))))
s.add(Implies(V[2][2], Not(Or(V[1][2], V[3][2], V[4][2]))))
s.add(Implies(V[2][3], Not(Or(V[1][3], V[3][3], V[4][3]))))

s.add(Or(V[3][1], V[3][2], V[3][3]))
s.add(Implies(V[3][1], Not(Or(V[1][1], V[2][1], V[4][1], V[5][1]))))
s.add(Implies(V[3][2], Not(Or(V[1][2], V[2][2], V[4][2], V[5][2]))))
s.add(Implies(V[3][3], Not(Or(V[1][3], V[2][3], V[4][3], V[5][3]))))

s.add(Or(V[4][1], V[4][2], V[4][3]))
s.add(Implies(V[4][1], Not(Or(V[2][1], V[3][1], V[5][1], V[6][1]))))
s.add(Implies(V[4][2], Not(Or(V[2][2], V[3][2], V[5][2], V[6][2]))))
s.add(Implies(V[4][3], Not(Or(V[2][3], V[3][3], V[5][3], V[6][3]))))

s.add(Or(V[5][1], V[5][2], V[5][3]))
s.add(Implies(V[5][1], Not(Or(V[3][1], V[4][1], V[6][1]))))
s.add(Implies(V[5][2], Not(Or(V[3][2], V[4][2], V[6][2]))))
s.add(Implies(V[5][3], Not(Or(V[3][3], V[4][3], V[6][3]))))

s.add(Or(V[6][1], V[6][2], V[6][3]))
s.add(Implies(V[6][1], Not(Or(V[1][1], V[4][1], V[5][1]))))
s.add(Implies(V[6][2], Not(Or(V[1][2], V[4][2], V[5][2]))))
s.add(Implies(V[6][3], Not(Or(V[1][3], V[4][3], V[5][3]))))

print(s.check())
print(s.model())
s.assertions()

sat
[s_2_2 = False,
 s_6_2 = False,
 s_4_3 = False,
 s_5_1 = False,
 s_4_2 = True,
 s_6_3 = False,
 s_1_2 = True,
 s_2_3 = True,
 s_3_2 = False,
 s_3_3 = False,
 s_5_2 = False,
 s_1_1 = False,
 s_5_3 = True,
 s_1_3 = False,
 s_4_1 = False,
 s_2_1 = False,
 s_3_1 = True,
 s_6_1 = True]


[Or(s_1_1, s_1_2, s_1_3),
 Implies(s_1_1, Not(Or(s_2_1, s_3_1, s_6_1))),
 Implies(s_1_2, Not(Or(s_2_2, s_3_2, s_6_2))),
 Implies(s_1_3, Not(Or(s_2_3, s_3_3, s_6_3))),
 Or(s_2_1, s_2_2, s_2_3),
 Implies(s_2_1, Not(Or(s_1_1, s_3_1, s_4_1))),
 Implies(s_2_2, Not(Or(s_1_2, s_3_2, s_4_2))),
 Implies(s_2_3, Not(Or(s_1_3, s_3_3, s_4_3))),
 Or(s_3_1, s_3_2, s_3_3),
 Implies(s_3_1, Not(Or(s_1_1, s_2_1, s_4_1, s_5_1))),
 Implies(s_3_2, Not(Or(s_1_2, s_2_2, s_4_2, s_5_2))),
 Implies(s_3_3, Not(Or(s_1_3, s_2_3, s_4_3, s_5_3))),
 Or(s_4_1, s_4_2, s_4_3),
 Implies(s_4_1, Not(Or(s_2_1, s_3_1, s_5_1, s_6_1))),
 Implies(s_4_2, Not(Or(s_2_2, s_3_2, s_5_2, s_6_2))),
 Implies(s_4_3, Not(Or(s_2_3, s_3_3, s_5_3, s_6_3))),
 Or(s_5_1, s_5_2, s_5_3),
 Implies(s_5_1, Not(Or(s_3_1, s_4_1, s_6_1))),
 Implies(s_5_2, Not(Or(s_3_2, s_4_2, s_6_2))),
 Implies(s_5_3, Not(Or(s_3_3, s_4_3, s_6_3))),
 Or(s_6_1, s_6_2, s_6_3),
 Implies(s_6_1, Not(Or(s_1_1, s_4_1, s_5_1))),
 Implies(s_6_2, Not(Or(s_1_2, s_4_2, s_5_2))),
 Implies(s_6_3, Not(Or(s_1_3, s_4_3, s_5_3)))]

## Z3 as SMT solver

Z3 can also be used to solve set of assertions using different domains. Below an example with integers, see [Z3Py Guide](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) for more.

### [Arithmetic](https://rise4fun.com/z3/tutorialcontent/guide#h24)

$$(x - y \leq 0) \land (y - z \leq 0) \land ((z - x \leq -1) \lor (z - x \leq -2))$$

In [15]:
from z3 import *

x, y, z = Ints('x y z')

s = Solver()

s.add((x - y) <= 0)
s.add((y - z) <= 0)
s.add((z - x) <= -1)
s.add((z - x) <= -2)

print(s.assertions())

if s.check() == sat:
    print(f'Solution: {s.model()}')
else:
    print("Unsatisfiable")

[x - y <= 0, y - z <= 0, z - x <= -1, z - x <= -2]
Unsatisfiable


$$(b \lor (x + y \leq 0)) \land (\neg b \lor (x + z \leq 10))$$

In [16]:
from z3 import *

x, y, z = Ints('x y z')
b = Bool('b')

s = Solver()

s.add(Or(b, (x + y) <= 0))
s.add(Or(Not(b), (x + z) <= 10))

print(s.assertions())

if s.check() == sat:
    print(f'Solution: {s.model()}')
else:
    print("Unsatisfiable")

[Or(b, x + y <= 0), Or(Not(b), x + z <= 10)]
Solution: [b = False, y = 0, z = 11, x = 0]


## Minesweeper

Let's consider the [Minesweeper](https://en.wikipedia.org/wiki/Minesweeper_(video_game)) game, where you have a grid of covered squares, some of which contain mines, but you don't know which. Your job is to uncover every square which does not contain a mine. If you uncover a square containing a mine, you lose. If you uncover a square which does not contain a mine, you are told how many mines are contained within the eight surrounding squares.

Once you start unveiling the minefield you can deduce the location of mines by solving a system of equations by trying to *disprove* that in a specific cell you can place a mine. In order to do that you can represent each cell by an integer variable that contains the number of mines in the specific cell (0 or 1).

For this exercise we will use a problem generator with the property that the generated problems can be always solved by deduction (mind that in general this is not the case). The generator is available on the [Simon Tatham's Portable Puzzle Collection](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html) page:

> The first square you open is guaranteed to be safe, and (by default) you are guaranteed to be able to solve the whole grid by deduction rather than guesswork. (Deductions may require you to think about the total number of mines.)

An interesting instance you can try to solve is the following:

[![sweeper example](media/sweeper_6x6-10.png)](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe)

and you can use [this link](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe) to verify your solution. You should include below the Python code you're using to solve the instance.

A convenient representation for the problem (useful later on) is a matrix of numbers representing the number of mines *seen* by the cell or '?' for the unknown cells. E.g the above example would correspond to

    ??????
    ?1123?
    ?1003?
    ?1002?
    ?2234?
    ??????

You can use the Sudoku example in [Z3Py Guide](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) as inspiration for your code. Note that solving the game might require several calls to the solver as well as the update of the facts you know about the board when you unveil more cells.

In [17]:
# Minesweeper
(t00, t10, t20, t30, t40, t50,
t01, t51,
t02, t52,
t03, t53,
t04, t54,
t05, t55,
t15, t25, t35, t45) = Ints('t00 t10 t20 t30 t40 t50 t01 t51 t02 t52 t03 t53 t04 t54 t05 t55 t15 t25 t35 t45')

s = Solver()

s.add(
    Or(t00 == 0, t00 == 1),
    Or(t10 == 0, t10 == 1),
    Or(t20 == 0, t20 == 1),
    Or(t30 == 0, t30 == 1),
    Or(t40 == 0, t40 == 1),
    Or(t50 == 0, t50 == 1),

    Or(t01 == 0, t01 == 1),
    Or(t51 == 0, t51 == 1),

    Or(t02 == 0, t02 == 1),
    Or(t52 == 0, t52 == 1),

    Or(t03 == 0, t03 == 1),
    Or(t53 == 0, t53 == 1),

    Or(t04 == 0, t04 == 1),
    Or(t54 == 0, t54 == 1),

    Or(t05 == 0, t05 == 1),
    Or(t15 == 0, t15 == 1),
    Or(t25 == 0, t25 == 1),
    Or(t35 == 0, t35 == 1),
    Or(t45 == 0, t45 == 1),
    Or(t55 == 0, t55 == 1),

    (   
        t00 + t10 + t20 + t30 + t40 + t50 + 
        t01 + t51 + 
        t02 + t52 + 
        t03 + t53 + 
        t04 + t54 + 
        t05 + t55 + 
        t15 + t25 + t35 + t45 + t55 == 10
    ),

    # Checking adjacent cells
    (t00 + t10 + t20 + t01 + t02 == 2),
    (t10 + t20 + t30 == 2),
    (t20 + t30 + t40 == 3),
    (t30 + t40 + t50 + t51 + t52 == 4),

    (t01 + t02 + t03 == 1),
    (t51 + t52 + t53 == 2),

    (t02 + t03 + t04 == 1),
    (t52 + t53 + t54 == 3),

    (t05 + t15 + t25 + t04 + t03 == 1),
    (t15 + t25 + t35 == 1),
    (t25 + t35 + t45 == 2),
    (t35 + t45 + t55 + t54 + t53 == 3)
)

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[t04 = 0,
 t45 = 1,
 t03 = 0,
 t53 = 1,
 t54 = 1,
 t10 = 0,
 t00 = 0,
 t55 = 0,
 t30 = 1,
 t50 = 1,
 t20 = 1,
 t05 = 0,
 t15 = 0,
 t25 = 1,
 t35 = 0,
 t01 = 0,
 t02 = 1,
 t40 = 1,
 t51 = 0,
 t52 = 1]


## Deduction in the Wumpus world

In this exercise you should consider the [Hunt the Wumpus](https://docs.google.com/document/d/1ySK0M-txOuIVWUGxPB02Ws_p5nSi5aLMlDQl4NMtAlk/edit?usp=sharing) domain, and use the solver to deduce facts about the given instance of the problem.

Use the techniques that you learned in the previous exercises to deduce (if possible) the safe cells and the position of pits and Wumpus given the perceptions that the agent would detect from a given cell.

The input is given as a matrix where cells contain either the letter `B` (breeze) or `S` (stench) according whether the agent, placed in the corresponding cell, would feel one of the two situations. We assume that the sensing information *doesn't* include the presence of pit and wumpus in the cells where they are located (see the example below for details).

E.g. a possible configuration and the corresponding "perception" matrix would be:

    +---+---+---+---+        +---+---+---+---+
    |  P| G |   |   |        |   |  B|S  |   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |W  |   |        |  B|S  |   |S B|
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+ =====> +---+---+---+---+
    |   |   |   |  P|        |   |   |S B|   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |   |   |        |   |   |   |  B|
    | A |   |   |   |        | A |   |   |   |
    +---+---+---+---+        +---+---+---+---+

Note that neither the cells where pits and wumpus are located don't sense breeze or stench coming from the same cell. They would sense neighbouring cells. 


Try to solve the following two examples by encoding using Z3 and explain the method you used.

1. **Example 1**
    ```
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|   |  B|S B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|S  |   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [18]:
# Wumpus 1
(p00, p10, p20, p30,
p01, p11, p21, p31,
p02, p12, p22, p32,
p03, p13, p23, p33,
w10, w30, w21, w32) = Ints('p00 p10 p20 p30 p01 p11 p21 p31 p02 p12 p22 p32 p03 p13 p23 p33 w10 w30 w21 w32')

s = Solver()

s.add(
    # Setting pit constraints for adjacent cells
    # Checking cells starting from bottom left, moving left to right, shifting upwards row-wise.
    (p00 == 0),
    Or(p10 == 0, p10 == 1),
    Or(p20 == 0, p20 == 1),
    Or(p30 == 0, p30 == 1),

    Or(p01 == 0, p01 == 1),
    Or(p11 == 0, p11 == 1),
    Or(p21 == 0, p21 == 1),
    Or(p31 == 0, p31 == 1),

    Or(p02 == 0, p02 == 1),
    Or(p12 == 0, p12 == 1),
    Or(p22 == 0, p22 == 1),
    Or(p32 == 0, p32 == 1),

    Or(p03 == 0, p03 == 1),
    Or(p13 == 0, p13 == 1),
    Or(p23 == 0, p23 == 1),
    Or(p33 == 0, p33 == 1),

    Or(w10 == 0, w10 == 1),
    Or(w30 == 0, w30 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w32 == 0, w32 == 1),

    (p10 + p01 == 0),
    And((p20 + p11) >= 1, (p20 + p11) <= 2),
    (p10 + p30 + p21 == 0),
    (p20 + p31 == 0),
    And((p11 + p02) >= 1, (p11 + p02) <= 2),
    (p10 + p01 + p21 + p12 == 0),
    And((p20 + p11 + p31 + p22) >= 1, (p20 + p11 + p31 + p22) <= 4),
    And((p30 + p21 + p32) >= 1, (p30 + p21 + p32) <= 3),
    (p01 + p12 + p03 == 0),
    And((p11 + p02 + p22 + p13) >= 1, (p11 + p02 + p22 + p13) <= 4),
    And((p21 + p12 + p32 + p23) >= 1, (p21 + p12 + p32 + p23) <= 4),
    And((p31 + p22 + p33) >= 1, (p31 + p22 + p33) <= 3),
    (p02 + p13 == 0),
    And((p12 + p03 + p23) >= 1, (p12 + p03 + p23) <= 3),
    And((p22 + p13 + p33) >= 1, (p22 + p13 + p33) <= 3),
    And((p32 + p23) >= 1, (p32 + p23) <= 2),

    # Setting wumpus constraints for adjacent cells
    Or(w10 == 0, w10 == 1),
    Or(w30 == 0, w30 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w32 == 0, w32 == 1),

    (w10 + w30 + w21 + w32 == 1),

    And(
        (w10 + w30 + w21 == 1),
        (w30 + w21 + w32 == 1)
    )
)

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[p22 = 0,
 p33 = 1,
 w30 = 0,
 p32 = 1,
 p11 = 1,
 p13 = 0,
 p03 = 0,
 p31 = 0,
 p01 = 0,
 p23 = 1,
 w32 = 0,
 w10 = 0,
 p21 = 0,
 p10 = 0,
 w21 = 1,
 p02 = 0,
 p20 = 0,
 p30 = 0,
 p12 = 0,
 p00 = 0]
[w30 = 1,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 0,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 0,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 1,
 w10 = 0,
 p13 = 0]
[w30 = 1,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 0,
 p31 = 0,
 p20 = 0,
 p22 = 0,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 1,
 w10 = 0,
 p13 = 0]
[w30 = 1,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 0,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 1,
 w10 = 0,
 p13 = 0]
[w30 = 0,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 1,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 1,
 w10 = 0,
 p13 = 0]


2. **Example 2**
    ```
    +---+---+---+---+
    |   |  B|S  |  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|S B|  B|S  |
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|S B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|  B|   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [19]:
# Wumpus 2
(p00, p10, p20, p30,
p01, p11, p21, p31,
p02, p12, p22, p32,
p03, p13, p23, p33,
w20,
w11, w21, w31,
w02, w12, w22, w32,
w13, w23, w33) = Ints('p00 p10 p20 p30 p01 p11 p21 p31 p02 p12 p22 p32 p03 p13 p23 p33 w20 w11 w21 w31 w02 w12 w22 w32 w13 w23 w33')

s = Solver()

s.add(
    # Setting pit constraints for adjacent cells
    # Checking cells starting from bottom left, moving left to right, shifting upwards row-wise.
    (p00 == 0),
    Or(p10 == 0, p10 == 1),
    Or(p20 == 0, p20 == 1),
    Or(p30 == 0, p30 == 1),

    Or(p01 == 0, p01 == 1),
    Or(p11 == 0, p11 == 1),
    Or(p21 == 0, p21 == 1),
    Or(p31 == 0, p31 == 1),

    Or(p02 == 0, p02 == 1),
    Or(p12 == 0, p12 == 1),
    Or(p22 == 0, p22 == 1),
    Or(p32 == 0, p32 == 1),

    Or(p03 == 0, p03 == 1),
    Or(p13 == 0, p13 == 1),
    Or(p23 == 0, p23 == 1),
    Or(p33 == 0, p33 == 1),

    Or(w10 == 0, w10 == 1),
    Or(w30 == 0, w30 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w32 == 0, w32 == 1),

    And((p10 + p01) >= 1, (p10 + p01) <= 2),
    And((p20 + p11) >= 1, (p20 + p11) <= 2),
    And((p10 + p30 + p21) >= 1, (p10 + p30 + p21) <= 3),
    (p20 + p31 == 0),
    And((p11 + p02) >= 1, (p11 + p02) <= 2),
    And((p10 + p01 + p21 + p12) >= 1, (p10 + p01 + p21 + p12) <= 4),
    And((p20 + p11 + p31 + p22) >= 1, (p20 + p11 + p31 + p22) <= 4),
    And((p30 + p21 + p32) >= 1, (p30 + p21 + p32) <= 3),
    And((p01 + p12 + p03) >= 1, (p01 + p12 + p03) <= 3),
    And((p11 + p02 + p22 + p13) >= 1, (p11 + p02 + p22 + p13) <= 4),
    And((p21 + p12 + p32 + p23) >= 1, (p21 + p12 + p32 + p23) <= 4),
    (p31 + p22 + p33 == 0),
    (p02 + p13 == 0),
    And((p12 + p03 + p23) >= 1, (p12 + p03 + p23) <= 3),
    (p22 + p13 + p33 == 0),
    And((p32 + p23) >= 1, (p32 + p23) <= 2),

    # Setting wumpus constraints for adjacent cells
    Or(w20 == 0, w20 == 1),
    Or(w11 == 0, w11 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w31 == 0, w31 == 1),
    Or(w02 == 0, w02 == 1),
    Or(w12 == 0, w12 == 1),
    Or(w22 == 0, w22 == 1),
    Or(w32 == 0, w32 == 1),
    Or(w13 == 0, w13 == 1),
    Or(w23 == 0, w23 == 1),
    Or(w33 == 0, w33 == 1),

    (w20 + w11 + w21 + w31 + w02 + w12 + w22 + w32 + w13 + w23 + w33 == 1),

    And(
        (w20 + w11 + w31 + w22 == 1),
        (w11 + w02 + w22 + w13 == 1),
        (w31 + w22 + w33 == 1),
        (w22 + w13 + w33 == 1)
    )
)

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[w10 = 0,
 w30 = 0,
 w13 = 0,
 p32 = 1,
 w33 = 0,
 p03 = 0,
 p23 = 0,
 w21 = 0,
 p33 = 0,
 w23 = 0,
 w32 = 0,
 w12 = 0,
 p01 = 1,
 p30 = 0,
 p10 = 0,
 p12 = 1,
 p11 = 1,
 p13 = 0,
 p21 = 1,
 w20 = 0,
 w02 = 0,
 w11 = 0,
 w31 = 0,
 w22 = 1,
 p02 = 0,
 p20 = 0,
 p31 = 0,
 p22 = 0,
 p00 = 0]
[p13 = 0,
 w33 = 0,
 p11 = 1,
 p10 = 1,
 p33 = 0,
 w22 = 1,
 p30 = 0,
 w20 = 0,
 p00 = 0,
 w21 = 0,
 w12 = 0,
 p20 = 0,
 p22 = 0,
 w02 = 0,
 p01 = 1,
 p12 = 1,
 w23 = 0,
 p32 = 1,
 w10 = 0,
 p21 = 0,
 w11 = 0,
 p03 = 0,
 p02 = 0,
 p23 = 0,
 w13 = 0,
 p31 = 0,
 w32 = 0,
 w31 = 0,
 w30 = 0]
[p13 = 0,
 w33 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 0,
 w22 = 1,
 p30 = 0,
 w20 = 0,
 p00 = 0,
 w21 = 0,
 w12 = 0,
 p20 = 0,
 p22 = 0,
 w02 = 0,
 p01 = 1,
 p12 = 1,
 w23 = 0,
 p32 = 1,
 w10 = 0,
 p21 = 1,
 w11 = 0,
 p03 = 0,
 p02 = 0,
 p23 = 0,
 w13 = 0,
 p31 = 0,
 w32 = 0,
 w31 = 0,
 w30 = 1]
[p13 = 0,
 w33 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 0,
 w22 = 1,
 p30 = 0,
 w20 = 0,
 p00 = 0,
 w21 = 0,
 w12 = 0,
 p20 = 0,
 p22 = 0,


Now you should consider the case that only partial information is available, i.e. for some cells you don't have any information (below they're marked with `?`). This means that you can have several possible configurations (i.e. *models* in logic terms) corresponding to the available informations.

You should encode the configuration below (it's the first example with some unknown cells) and verify whether the first model the solver returns is the same as the one found for the first example.

```
+---+---+---+---+
|   |  B|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|  ?|  B|
|   |   |   |   |
+---+---+---+---+
|  B|  ?|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|S  |   |
| A |   |   |   |
+---+---+---+---+
```


In [20]:
# Wumpus partial info
(p00, p10, p20, p30,
p01, p11, p21, p31,
p02, p12, p22, p32,
p03, p13, p23, p33,
w10, w30, w21, w32) = Ints('p00 p10 p20 p30 p01 p11 p21 p31 p02 p12 p22 p32 p03 p13 p23 p33 w10 w30 w21 w32')

p00 = Int("p00")
p10 = Int("p10")
p20 = Int("p20")
p30 = Int("p30")

p01 = Int("p01")
p11 = Int("p11")
p21 = Int("p21")
p31 = Int("p31")

p02 = Int("p02")
p12 = Int("p12")
p22 = Int("p22")
p32 = Int("p32")

p03 = Int("p03")
p13 = Int("p13")
p23 = Int("p23")
p33 = Int("p33")

w10 = Int("w10")
w30 = Int("w30")
w21 = Int("w21")
w32 = Int("w32")

s = Solver()

s.add(
    # Setting pit constraints for adjacent cells
    # Checking cells starting from bottom left, moving left to right, shifting upwards row-wise.
    (p00 == 0),
    Or(p10 == 0, p10 == 1),
    Or(p20 == 0, p20 == 1),
    Or(p30 == 0, p30 == 1),

    Or(p01 == 0, p01 == 1),
    Or(p11 == 0, p11 == 1),
    Or(p21 == 0, p21 == 1),
    Or(p31 == 0, p31 == 1),

    Or(p02 == 0, p02 == 1),
    Or(p12 == 0, p12 == 1),
    Or(p22 == 0, p22 == 1),
    Or(p32 == 0, p32 == 1),

    Or(p03 == 0, p03 == 1),
    Or(p13 == 0, p13 == 1),
    Or(p23 == 0, p23 == 1),
    Or(p33 == 0, p33 == 1),

    Or(w10 == 0, w10 == 1),
    Or(w30 == 0, w30 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w32 == 0, w32 == 1),

    (p10 + p01 == 0),
    And((p20 + p11) >= 1, (p20 + p11) <= 2),
    (p10 + p30 + p21 == 0),
    (p20 + p31 == 0),
    And((p11 + p02) >= 1, (p11 + p02) <= 2),
    #(p10 + p01 + p21 + p12 == 0),
    #And((p20 + p11 + p31 + p22) >= 1, (p20 + p11 + p31 + p22) <= 4),
    #And((p30 + p21 + p32) >= 1, (p30 + p21 + p32) <= 3),
    (p01 + p12 + p03 == 0),
    And((p11 + p02 + p22 + p13) >= 1, (p11 + p02 + p22 + p13) <= 4),
    #And((p21 + p12 + p32 + p23) >= 1, (p21 + p12 + p32 + p23) <= 4),
    And((p31 + p22 + p33) >= 1, (p31 + p22 + p33) <= 3),
    (p02 + p13 == 0),
    And((p12 + p03 + p23) >= 1, (p12 + p03 + p23) <= 3),
    #And((p22 + p13 + p33) >= 1, (p22 + p13 + p33) <= 3),
    #And((p32 + p23) >= 1, (p32 + p23) <= 2),

    # Setting wumpus constraints for adjacent cells
    Or(w10 == 0, w10 == 1),
    Or(w30 == 0, w30 == 1),
    Or(w21 == 0, w21 == 1),
    Or(w32 == 0, w32 == 1),

    (w10 + w30 + w21 + w32 == 1),
    (w10 + w30 + w21 == 1)
)

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[p33 = 0,
 w10 = 0,
 p31 = 0,
 p12 = 0,
 p32 = 0,
 p11 = 1,
 p21 = 0,
 w21 = 0,
 p13 = 0,
 p03 = 0,
 p22 = 1,
 p23 = 1,
 w32 = 0,
 w30 = 1,
 p02 = 0,
 p20 = 0,
 p30 = 0,
 p10 = 0,
 p01 = 0,
 p00 = 0]
[w30 = 0,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 0,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 0,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 0,
 w10 = 1,
 p13 = 0]
[w30 = 0,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 0,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 0,
 w10 = 1,
 p13 = 0]
[w30 = 0,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 1,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 0,
 w10 = 0,
 p13 = 0]
[w30 = 0,
 p21 = 0,
 p11 = 1,
 p10 = 0,
 p33 = 1,
 p03 = 0,
 p30 = 0,
 p02 = 0,
 p23 = 1,
 p00 = 0,
 w21 = 1,
 p31 = 0,
 p20 = 0,
 p22 = 1,
 w32 = 0,
 p01 = 0,
 p12 = 0,
 p32 = 1,
 w10 = 0,
 p13 = 0]


## Example: enumerating all models

Using the API you can write a procedure to enumerate all the models. The key to avoid the repetition of models that have been already enumerated is to add a formulae that falsify the models already discovered in such a way that they would be avoided in the search.

Consider that a model is a set of variable assignments; therefore a formula that is blocking the model is the negation of the value assignments; e.g.:

In [21]:
from z3 import *
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Xor(Tie, Shirt))

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[Tie = True, Shirt = False]
[Tie = False, Shirt = True]


Note the `v()` reference to the variable from the model and be aware that the above technique doesn't work with functions and arrays, see [this](https://stackoverflow.com/a/11869410) stackoverflow answer for details. A more advanced technique is described in [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-blocking-evaluations).